# Parsing a CSGO demofile¶
##### Last Updated: October 31, 2022

The awpy package allows for the parsing of CSGO demofiles into JSON or tabular format, which makes for easy analysis. To install the package, clone the Github repository, navigate to the root directory and run `python setup.py install`. Make sure you have installed at least [Golang](https://golang.org/) version 1.17. Please see expanded documentation [here](https://github.com/pnxenopoulos/csgo/tree/main/csgo/docs).

##### What is a demofile?
Each CSGO match typically generates recordings of the game called a demofile. Every map in a match will generate its own demofile. This demofile contains a serialization of the data transferred between the host (game server) and its clients (players). Data transferred to and from the server occurs at a predefined tick rate, which defines when client inputs are resolved with the server. For professional games, the server tick rate is usually 128 ticks per second, meaning each tick represents around 7.8 milliseconds. Client inputs represent player actions, such as movement, attacks or grenade throws. Non-client events also register in the demofile, such as round starts and ends.

Since CSGO demofiles are essentially data transferred between the clients and the game server, the data is simply stored as a text of sequential set of events with no contextual information, such as round or map location. Thus, due to the highly unstructured nature of these low level CSGO data streams, akin to log files, performing any complex analytic tasks becomes impossible without modeling this data formally into a useful data structure. The awpy package addresses this need.

##### How do I get demofiles?¶
One of the most common ways to obtain demofiles is through the site [HLTV](https://hltv.org). If you navigate to a match page, an hour or so after the match has concluded, it will often contain a link to the demofiles for that match. The link is a `.rar` compressed directory containing a demo for each map in the match.

# Initializing the parser
Let's consider the demo from a match between OG and Natus Vincere (NaVi), which we can [download from HLTV](https://www.hltv.org/matches/2344822/og-vs-natus-vincere-blast-premier-fall-series-2020). If we download the compressed demofile directory (a few hundred MB), there are three files:

- `og-vs-natus-vincere-m1-dust2.dem`
- `og-vs-natus-vincere-m2-mirage.dem`
- `og-vs-natus-vincere-m3-inferno.dem`

In order to parse one of the demofiles, you have to instantiate a DemoParser object and pass it the path to the demofile. The parser can automatically infer a `demo_id` from the filename, but you can also specify one using the argument. This name is use to title the JSON output file from the parser. We will use a `demo_id` of `OG-NaVi-BLAST2020`. 

There are a few numeric arguments to specify as one. The primary one is the `parse_rate`, or, how often do we want to parse a frame? This does _not_ affect how often we parse events, like kills or damages, only player positioning. Setting a parse rate of 1 means we are acquiring every frame (this is _very_ slow). Usually, 128 or 64 will work, depending on the tick rate of the demo. You can also specify the trade time (how many seconds inbetween trades) using the `trade_time` argument. Usually, a few seconds is good -- the default is 5. Finally, you can specify the round buy style (either "csgo" or "hltv", default is "hltv"). This argument changes the string for the round buy type output.

The parser writes to disk a file named `demoId.json`.

In [1]:
from awpy.parser import DemoParser

# Create parser object
# Set log=True above if you want to produce a logfile for the parser
demo_parser = DemoParser(
    demofile = "og-vs-natus-vincere-m1-dust2.dem", 
    demo_id = "OG-NaVi-BLAST2020", 
    parse_rate=128, 
    trade_time=5, 
    buy_style="hltv"
)


# Parse the demofile, output results to dictionary
data = demo_parser.parse()

# Accessing the parsed data
Next, we can access our parsed data. The default data output is the JSON-like format of a Python dictionary. We describe the structure in the [documentation](https://github.com/pnxenopoulos/csgo/tree/main/csgo/docs). We show the results in the following cells.

In [2]:
for k in data:
    if k != "gameRounds":
        print(k + ": " + str(data[k]))

matchID: OG-NaVi-BLAST2020
clientName: GOTV Demo
mapName: de_dust2
tickRate: 128
playbackTicks: 466670
playbackFramesCount: 466095
parsedToFrameIdx: 466110
parserParameters: {'parseRate': 128, 'parseFrames': False, 'parseKillFrames': False, 'tradeTime': 5, 'roundBuyStyle': 'hltv', 'damagesRolledUp': False}
serverVars: {'cashBombDefused': 0, 'cashBombPlanted': 0, 'cashTeamTWinBomb': 0, 'cashWinDefuse': 3500, 'cashWinTimeRunOut': 0, 'cashWinElimination': 0, 'cashPlayerKilledDefault': 0, 'cashTeamLoserBonus': 0, 'cashTeamLoserBonusConsecutive': 0, 'roundTime': 0, 'roundTimeDefuse': 2, 'roundRestartDelay': 5, 'freezeTime': 20, 'buyTime': 20, 'bombTimer': 0, 'maxRounds': 30, 'timeoutsAllowed': 4, 'coachingAllowed': 1}
matchPhases: {'announcementLastRoundHalf': [239899], 'announcementFinalRound': [], 'announcementMatchStarted': [], 'roundStarted': [890, 7167, 9177, 9308, 43817, 60608, 73511, 84662, 100420, 118340, 135431, 153351, 169400, 181861, 192298, 209674, 227594, 239899, 259186, 294862

## Event Data
The parser contains data on kills, damages, grenades, bomb events, weapon fires and flashes, which we show below, in order.

In [3]:
for k in data["gameRounds"][7]["kills"][0]:
        print(k + ": " + str(data["gameRounds"][7]["kills"][0][k]))

tick: 147660
seconds: 75.5390625
clockTime: 00:40
attackerSteamID: 76561198013243326
attackerName: Aleksib
attackerTeam: OGEsports
attackerSide: CT
attackerX: -1666.5416259765625
attackerY: 2220.43798828125
attackerZ: -3.0230464935302734
attackerViewX: 357.8961181640625
attackerViewY: 3.4881591796875
victimSteamID: 76561198146207066
victimName: Boombl4
victimTeam: Natus Vincere
victimSide: T
victimX: 179.58131408691406
victimY: 2156.62890625
victimZ: -127.86104583740234
victimViewX: 184.493408203125
victimViewY: 356.81396484375
assisterSteamID: None
assisterName: None
assisterTeam: None
assisterSide: None
isSuicide: False
isTeamkill: False
isWallbang: False
penetratedObjects: 0
isFirstKill: True
isHeadshot: True
victimBlinded: False
attackerBlinded: False
flashThrowerSteamID: None
flashThrowerName: None
flashThrowerTeam: None
flashThrowerSide: None
noScope: False
thruSmoke: False
distance: 1851.4389092968922
isTrade: False
playerTradedName: None
playerTradedTeam: None
playerTradedSteam

In [4]:
for k in data["gameRounds"][7]["damages"][0]:
        print(k + ": " + str(data["gameRounds"][7]["damages"][0][k]))

tick: 139558
seconds: 12.2421875
clockTime: 01:43
attackerSteamID: 76561197960710573
attackerName: NBK-
attackerTeam: OGEsports
attackerSide: CT
attackerX: 1293.0718994140625
attackerY: 671.2274169921875
attackerZ: -30.118175506591797
attackerViewX: 166.0528564453125
attackerViewY: 357.703857421875
attackerStrafe: False
victimSteamID: 76561198116523276
victimName: flamie
victimTeam: Natus Vincere
victimSide: T
victimX: 639.7489624023438
victimY: 516.0263061523438
victimZ: -1.3177638053894043
victimViewX: 290.8245849609375
victimViewY: 5.240478515625
weapon: Incendiary Grenade
weaponClass: Grenade
hpDamage: 1
hpDamageTaken: 1
armorDamage: 0
armorDamageTaken: 0
hitGroup: Generic
isFriendlyFire: False
distance: 672.1217959162599
zoomLevel: 0


In [5]:
for k in data["gameRounds"][7]["grenades"][0]:
        print(k + ": " + str(data["gameRounds"][7]["grenades"][0][k]))

throwTick: 138098
destroyTick: 140655
throwSeconds: 0.8359375
throwClockTime: 01:55
destroySeconds: 20.8125
destroyClockTime: 01:35
throwerSteamID: 76561198016432560
throwerName: mantuu
throwerTeam: OGEsports
throwerSide: CT
throwerX: 185.5625
throwerY: 2329.4375
throwerZ: -55.0625
grenadeType: Smoke Grenade
grenadeX: -370.4375
grenadeY: 2254.0625
grenadeZ: -119
entityId: 663172221523735513


In [6]:
for k in data["gameRounds"][6]["bombEvents"][0]:
        print(k + ": " + str(data["gameRounds"][6]["bombEvents"][0][k]))

tick: 129153
seconds: 64.4765625
clockTime: 00:51
playerSteamID: 76561198034202275
playerName: s1mple
playerTeam: Natus Vincere
playerX: 1089.201904296875
playerY: 2489.580810546875
playerZ: 96.02377319335938
bombAction: plant_begin
bombSite: A


In [7]:
for k in data["gameRounds"][7]["weaponFires"][0]:
        print(k + ": " + str(data["gameRounds"][7]["weaponFires"][30][k]))

tick: 147660
seconds: 75.5390625
clockTime: 00:40
playerSteamID: 76561198013243326
playerName: Aleksib
playerTeam: OGEsports
playerSide: CT
playerX: -1666.5416259765625
playerY: 2220.43798828125
playerZ: -3.0230464935302734
playerViewX: 357.8961181640625
playerViewY: 3.4881591796875
playerStrafe: False
weapon: Desert Eagle
weaponClass: Pistols
ammoInMagazine: 6
ammoInReserve: 35
zoomLevel: 0


In [8]:
for k in data["gameRounds"][7]["flashes"][0].keys():
        print(k + ": " + str(data["gameRounds"][7]["flashes"][0][k]))

tick: 138864
seconds: 6.8203125
clockTime: 01:49
attackerSteamID: 76561198116523276
attackerName: flamie
attackerTeam: Natus Vincere
attackerSide: T
attackerX: 607.80712890625
attackerY: 247.66790771484375
attackerZ: 0.42551231384277344
attackerViewX: 76.9317626953125
attackerViewY: 4.4659423828125
playerSteamID: 76561198037812456
playerName: ISSAA
playerTeam: OGEsports
playerSide: CT
playerX: 1482.4324951171875
playerY: 1590.5972900390625
playerZ: 56.256004333496094
playerViewX: 42.2589111328125
playerViewY: 16.5069580078125
flashDuration: 0.63357888


## Event Data as DataFrames

We can also parse the data into dataframes.

In [9]:
df = demo_parser.parse(return_type="df")
df.keys()

dict_keys(['matchID', 'clientName', 'mapName', 'tickRate', 'playbackTicks', 'rounds', 'kills', 'damages', 'grenades', 'flashes', 'weaponFires', 'bombEvents', 'frames', 'playerFrames'])

In [10]:
df["kills"]

,tick,seconds,clockTime,attackerSteamID,attackerName,attackerTeam,attackerSide,attackerX,attackerY,attackerZ,...,isTrade,playerTradedName,playerTradedTeam,playerTradedSteamID,playerTradedSide,weapon,weaponClass,roundNum,matchID,mapName
0,39106,16.117188,01:39,76561198016432560,mantuu,OGEsports,CT,-294.155731,626.777832,1.016636,...,False,None,None,NaN,None,USP-S,Pistols,1,OG-NaVi-BLAST2020,de_dust2
1,39549,19.578125,01:36,76561198037812448,ISSAA,OGEsports,CT,-1845.398804,2719.539307,32.390869,...,False,None,None,NaN,None,USP-S,Pistols,1,OG-NaVi-BLAST2020,de_dust2
2,39632,20.226562,01:35,76561198016432560,mantuu,OGEsports,CT,-171.508743,761.183167,2.132068,...,False,None,None,NaN,None,USP-S,Pistols,1,OG-NaVi-BLAST2020,de_dust2
3,39696,20.726562,01:35,76561198116523280,flamie,Natus Vincere,T,-604.332642,1334.031250,-108.254135,...,False,None,None,NaN,None,Glock-18,Pistols,1,OG-NaVi-BLAST2020,de_dust2
4,40045,23.453125,01:32,76561198016432560,mantuu,OGEsports,CT,-149.031250,1001.231262,1.489925,...,True,Aleksib,OGEsports,7.656120e+16,CT,USP-S,Pistols,1,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,456139,4.148438,00:36,76561198044045104,electronic,Natus Vincere,CT,-484.418335,1847.642334,-123.553299,...,False,None,None,NaN,None,M4A4,Rifle,25,OG-NaVi-BLAST2020,de_dust2
159,458062,19.171875,00:21,76561198114929872,valde,OGEsports,T,-2025.999146,1511.807251,33.910797,...,False,None,None,NaN,None,AK-47,Rifle,25,OG-NaVi-BLAST2020,de_dust2
160,458223,20.429688,00:20,76561198114929872,valde,OGEsports,T,-2015.828979,1507.713623,33.111588,...,False,None,None,NaN,None,AK-47,Rifle,25,OG-NaVi-BLAST2020,de_dust2
161,458294,20.984375,00:20,76561198044045104,electronic,Natus Vincere,CT,-1753.573730,1118.690674,31.989418,...,True,Boombl4,Natus Vincere,7.656120e+16,CT,M4A4,Rifle,25,OG-NaVi-BLAST2020,de_dust2


In [11]:
df["damages"]

,tick,seconds,clockTime,attackerSteamID,attackerName,attackerTeam,attackerSide,attackerX,attackerY,attackerZ,...,hpDamageTaken,armorDamage,armorDamageTaken,hitGroup,isFriendlyFire,distance,zoomLevel,roundNum,matchID,mapName
0,39015,15.406250,01:40,76561198016432560,mantuu,OGEsports,CT,-284.767151,632.744629,1.266963,...,32,0,0,Neck,False,465.277175,0.0,1,OG-NaVi-BLAST2020,de_dust2
1,39072,15.851562,01:40,76561198016432560,mantuu,OGEsports,CT,-294.155731,626.777832,1.016636,...,31,0,0,RightArm,False,486.924484,0.0,1,OG-NaVi-BLAST2020,de_dust2
2,39106,16.117188,01:39,76561198016432560,mantuu,OGEsports,CT,-294.155731,626.777832,1.016636,...,37,0,0,Head,False,481.229305,0.0,1,OG-NaVi-BLAST2020,de_dust2
3,39526,19.398438,01:36,76561198044045104,electronic,Natus Vincere,T,-579.786072,1494.131836,-110.432953,...,91,0,0,Head,False,844.723217,0.0,1,OG-NaVi-BLAST2020,de_dust2
4,39549,19.578125,01:36,76561198037812448,ISSAA,OGEsports,CT,-1845.398804,2719.539307,32.390869,...,100,0,0,Head,False,1451.886671,0.0,1,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,458191,20.179688,00:20,76561198146207072,Boombl4,Natus Vincere,CT,-1873.927979,1302.483154,33.934990,...,27,4,4,Chest,False,259.550535,0.0,25,OG-NaVi-BLAST2020,de_dust2
700,458217,20.382812,00:20,76561198146207072,Boombl4,Natus Vincere,CT,-1883.084595,1294.227783,32.957306,...,27,4,4,Chest,False,252.194264,0.0,25,OG-NaVi-BLAST2020,de_dust2
701,458223,20.429688,00:20,76561198114929872,valde,OGEsports,T,-2015.828979,1507.713623,33.111588,...,100,16,16,Head,False,251.518662,0.0,25,OG-NaVi-BLAST2020,de_dust2
702,458294,20.984375,00:20,76561198044045104,electronic,Natus Vincere,CT,-1753.573730,1118.690674,31.989418,...,46,19,19,Head,False,467.218752,0.0,25,OG-NaVi-BLAST2020,de_dust2
